# Import thư viện

In [ ]:
%pip install xgboost

In [ ]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np
import scipy.stats as stats
from itertools import combinations
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler, Normalizer, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-2-5dbbcb8ddf9f>:28: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# Đọc dữ liệu

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

folder = "/content/drive/MyDrive/IE224-DataAnalysis"

Mounted at /content/drive/


In [ ]:
df = pd.read_csv(folder + "/data_preprocessing_labelencoder.csv", index_col=0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6636 entries, 0 to 6635
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pc_brand             6636 non-null   int64  
 1   pc_model             6636 non-null   int64  
 2   elt_condition        6636 non-null   int64  
 3   elt_warranty         6636 non-null   int64  
 4   desktop_screen_size  6636 non-null   int64  
 5   pc_cpu               6636 non-null   int64  
 6   pc_ram               6636 non-null   int64  
 7   pc_vga               6636 non-null   int64  
 8   pc_drive_capacity    6636 non-null   int64  
 9   elt_origin           6636 non-null   int64  
 10  classify             6636 non-null   int64  
 11  price (VND)          6636 non-null   float64
 12  pc_cpu_label         6636 non-null   int64  
dtypes: float64(1), int64(12)
memory usage: 725.8 KB


In [ ]:
df.head()

,pc_brand,pc_model,elt_condition,elt_warranty,desktop_screen_size,pc_cpu,pc_ram,pc_vga,pc_drive_capacity,elt_origin,classify,price (VND),pc_cpu_label
0,15,101,1,5,7,14,8,1,3,6,0,13500000.0,2
1,15,101,1,6,7,9,6,2,6,3,0,1800000.0,2
2,15,101,1,6,7,14,3,1,3,3,0,8500000.0,2
3,15,101,0,6,7,14,8,1,3,6,0,12500000.0,2
4,15,101,1,2,8,8,6,1,1,3,0,6300000.0,2


# Chia tập train-test

In [ ]:
filter_df = df[df['price (VND)'] < 65000000]

In [ ]:
X = filter_df.drop(columns=['price (VND)']).to_numpy()
y = filter_df['price (VND)'].to_numpy()

In [ ]:
print('X shape: ', X.shape)
print('y shape: ', y.shape)

X shape:  (6622, 12)
y shape:  (6622,)


# Train và đánh giá model

In [ ]:
pipelines = {
    'multi': [('model', LinearRegression())],
    'multi_norm': [('scale', Normalizer()),
                    ('model', LinearRegression())],
    'multi_mmscale': [('scale', MinMaxScaler()),
                    ('model', LinearRegression())],
    'multi_stscale': [('scale', StandardScaler()),
                    ('model', LinearRegression())],
    'Random Forest': [('model', RandomForestRegressor())],
    'Decision Tree': [('model', DecisionTreeRegressor())],
    'SVR': [('model', SVR())],
    'Histogram Gradient Boosting' : [('model', HistGradientBoostingRegressor())],
    'XGBoost': [('model', XGBRegressor())],
    'Neural Network': [('model', MLPRegressor())],
    'KNN': [('model', KNeighborsRegressor())]
}

In [ ]:
results = []

for name, pipeline in pipelines.items():
  model = Pipeline(pipeline)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  r2 = r2_score(y_test, y_pred)
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  results.append({
        'model': name,
        'r2': r2,
        'mse': mse,
        'mae': mae,
    })

pd.DataFrame(results).sort_values(by=['r2', 'mae', 'mse'],
                                  ascending=[False, True, True]).reset_index(drop=True)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,model,r2,mse,mae
0,Histogram Gradient Boosting,0.709438,1.214390e+13,2.247683e+06
1,XGBoost,0.704299,1.235868e+13,2.246087e+06
2,Random Forest,0.642799,1.492903e+13,2.440168e+06
3,KNN,0.545041,1.901478e+13,2.673050e+06
4,multi_mmscale,0.261507,3.086497e+13,3.723290e+06
5,multi_stscale,0.261507,3.086497e+13,3.723290e+06
6,multi,0.261507,3.086497e+13,3.723290e+06
7,Decision Tree,0.244301,3.158409e+13,3.116065e+06
8,multi_norm,0.221013,3.255738e+13,3.879834e+06
9,SVR,-0.050502,4.390522e+13,4.175250e+06


# Fine Tuning

In [ ]:
param_grid = {
    'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.0002, 0.002, 0.02, 0.2 , 0.0005, 0.005, 0.05, 0.5],
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'l2_regularization': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'max_leaf_nodes':[20, 25, 30, 35, 40, 45],
    'min_samples_leaf':[10, 15, 20, 25 ,30],
    'early_stopping': [True, False],
    'warm_start': [True, False]
}

clf = RandomizedSearchCV(HistGradientBoostingRegressor(), param_grid, n_iter=500 , verbose=3)
search = clf.fit(X_train, y_train)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV 1/5] END early_stopping=False, l2_regularization=0.3, learning_rate=0.0002, max_depth=5, max_leaf_nodes=40, min_samples_leaf=10, warm_start=False;, score=0.021 total time=   0.7s
[CV 2/5] END early_stopping=False, l2_regularization=0.3, learning_rate=0.0002, max_depth=5, max_leaf_nodes=40, min_samples_leaf=10, warm_start=False;, score=0.022 total time=   0.4s
[CV 3/5] END early_stopping=False, l2_regularization=0.3, learning_rate=0.0002, max_depth=5, max_leaf_nodes=40, min_samples_leaf=10, warm_start=False;, score=0.020 total time=   0.3s
[CV 4/5] END early_stopping=False, l2_regularization=0.3, learning_rate=0.0002, max_depth=5, max_leaf_nodes=40, min_samples_leaf=10, warm_start=False;, score=0.021 total time=   0.3s
[CV 5/5] END early_stopping=False, l2_regularization=0.3, learning_rate=0.0002, max_depth=5, max_leaf_nodes=40, min_samples_leaf=10, warm_start=False;, score=0.022 total time=   0.3s
[CV 1/5] END early_st

In [ ]:
y_pred = search.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

In [ ]:
print('R2:', r2)
print('MSE:' , mse)
print('MAE:', mae)

R2: 0.7188616525480944
MSE: 11750040359137.404
MAE: 2253531.969568334


In [ ]:
search.best_params_

{'warm_start': False,
 'min_samples_leaf': 10,
 'max_leaf_nodes': 20,
 'max_depth': 9,
 'learning_rate': 0.1,
 'l2_regularization': 0.0,
 'early_stopping': False}